## Preparando o Ambiente

In [ ]:
#Colab imports
from google.colab import drive
drive.mount('/content/drive')
#pd.set_option('display.max_columns', None)
#!pip install MiniSom

#Basic imports
import numpy as np
import pandas as pd
from random import *
import re

#Sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'

#Graphic imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Mounted at /content/drive


lendo o arquivo

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/UFAM/6o_periodo/IA/TP2/Data/StartUpsESG_0602.xlsx")

## Executando os passos do Visualizing High Dimensional Clusters

### Preparando os dataframes

In [ ]:
#visualização
df

,id,StartUp,Country,Year,ESG,E,S,G,Raised,distr_tk,...,FISCALFRD,FINACIFRD,FOREIGINVEST,FOREIGDP,INTERPAY,percmath,GINI,MAKTEFF,PERCTECH,DEVCOUNT
0,12316,VenusEnergy,Lithuania,2018,0.010,0.008,0.002,0.000,5986183.0,0.27,...,96.7,70,1.299842e+09,2.420713,3.476571,4.26941,35.7,4.642995,3.11263,1
1,23416,GreenEnergyCoin,Netherlands,2018,0.018,0.010,0.006,0.002,0.0,0.00,...,88.2,80,-3.614670e+11,-39.565294,2.264509,6.37706,28.1,5.408080,2.82506,1
2,1553,HydroMiner,Austria,2017,0.014,0.010,0.000,0.004,3500000.0,0.00,...,81.1,70,-2.855524e+10,-6.277635,3.644302,6.07867,30.8,4.897589,4.38647,1
3,17028,PowerCorp,"Hong Kong SAR, China",2018,0.008,0.006,0.000,0.002,0.0,0.00,...,100.0,90,9.703626e+10,26.828457,0.050000,0.00000,0.0,5.705542,0.00000,1
4,8483,Emyron Coin,United States,2018,0.017,0.000,0.011,0.006,0.0,0.28,...,54.8,80,2.614820e+11,1.268600,11.816379,7.20920,41.4,5.209139,3.78213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,6422,SUPP Network,United States,0,0.000,0.000,0.000,0.000,0.0,0.30,...,54.8,80,2.614820e+11,1.268600,11.816379,7.20920,41.4,5.209139,3.78213,1
6537,6423,Thousand Token,Colombia,0,0.000,0.000,0.000,0.000,0.0,0.10,...,82.2,70,1.153511e+10,3.451576,9.907686,1.38160,50.4,4.063790,4.82347,0
6538,6424,Cryptocurrency Market,Spain,0,0.000,0.000,0.000,0.000,0.0,0.70,...,36.1,70,5.538257e+10,3.896177,6.713329,5.19562,34.7,4.451561,3.92247,0
6539,6425,IQ Token,Belarus,0,0.000,0.000,0.000,0.000,0.0,0.00,...,75.4,10,1.426500e+09,2.376262,6.228334,2.84452,25.2,0.000000,4.51118,0


Tirando uma cópia do arquivo para manipulação

In [ ]:
X = df.copy(deep = True)
X = X[['Country','Raised', 'ESG', 'E', 'S','G']]
X

,Country,Raised,ESG,E,S,G
0,Lithuania,5986183.0,0.010,0.008,0.002,0.000
1,Netherlands,0.0,0.018,0.010,0.006,0.002
2,Austria,3500000.0,0.014,0.010,0.000,0.004
3,"Hong Kong SAR, China",0.0,0.008,0.006,0.000,0.002
4,United States,0.0,0.017,0.000,0.011,0.006
...,...,...,...,...,...,...
6536,United States,0.0,0.000,0.000,0.000,0.000
6537,Colombia,0.0,0.000,0.000,0.000,0.000
6538,Spain,0.0,0.000,0.000,0.000,0.000
6539,Belarus,0.0,0.000,0.000,0.000,0.000


Verificando nulos (nada na saída = ok)

In [ ]:
for col, hasNull in X.isnull().any().items():
  if(hasNull):
    print(f'coluna{col}:  {hasNull}')

### Usando o método get_dummies do pandas para converter colunas não numéricas.

In [ ]:
#We use pandas's 'get_dummies()' method
X = pd.get_dummies(X)

In [ ]:
X.head()

,Raised,ESG,E,S,G,Country_Afghanistan,Country_Albania,Country_Argentina,Country_Armenia,Country_Australia,...,Country_Tunisia,Country_Turkey,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States,"Country_Venezuela, RB",Country_Vietnam,Country_Zimbabwe,Country_india
0,5986183.0,0.010,0.008,0.002,0.000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.018,0.010,0.006,0.002,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3500000.0,0.014,0.010,0.000,0.004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.008,0.006,0.000,0.002,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.017,0.000,0.011,0.006,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Clustering

### Separando as colunas numéricas originais das colunas não numéricas convertidas
No site, o autor separa manualmente. Aqui, fiz um pequeno código

In [ ]:
numer = []
cater = []

for col in X.columns:
  if(X[col].dtype == np.int64 or X[col].dtype == np.float64):
    numer.append(col)
  else:
    cater.append(col)

print('numericas originais:',numer)
print('nao numericas convertidas:', cater)

numericas originais: ['Raised', 'ESG', 'E', 'S', 'G']
nao numericas convertidas: ['Country_Afghanistan', 'Country_Albania', 'Country_Argentina', 'Country_Armenia', 'Country_Australia', 'Country_Austria', 'Country_Azerbaijan', 'Country_Bahamas, The', 'Country_Bangladesh', 'Country_Belarus', 'Country_Belgium', 'Country_Belize', 'Country_Bosnia and Herzegovina', 'Country_Botswana', 'Country_Brazil', 'Country_British Virgin Islands', 'Country_Bulgaria', 'Country_Cambodia', 'Country_Cameroon', 'Country_Canada', 'Country_Cayman Islands', 'Country_Chile', 'Country_China', 'Country_Colombia', 'Country_Costa Rica', 'Country_Croatia', 'Country_Cyprus', 'Country_Czech Republic', 'Country_Denmark', 'Country_Dominican Republic', 'Country_Ecuador', 'Country_Egypt, Arab Rep.', 'Country_Estonia', 'Country_Finland', 'Country_France', 'Country_Georgia', 'Country_Germany', 'Country_Ghana', 'Country_Gibraltar', 'Country_Greece', 'Country_Guinea-Bissau', 'Country_Hong Kong SAR, China', 'Country_Hungary', '

In [ ]:
# gerando um dataframe apenas com as colunas numericas originais
numer = X.loc[:, numer].copy(deep = True)
numer

,Raised,ESG,E,S,G
0,5986183.0,0.010,0.008,0.002,0.000
1,0.0,0.018,0.010,0.006,0.002
2,3500000.0,0.014,0.010,0.000,0.004
3,0.0,0.008,0.006,0.000,0.002
4,0.0,0.017,0.000,0.011,0.006
...,...,...,...,...,...
6536,0.0,0.000,0.000,0.000,0.000
6537,0.0,0.000,0.000,0.000,0.000
6538,0.0,0.000,0.000,0.000,0.000
6539,0.0,0.000,0.000,0.000,0.000


In [ ]:
# gerando um dataframe apenas com as colunas não numericas convertidas
cater = X.loc[:, cater].copy(deep = True)
cater

,Country_Afghanistan,Country_Albania,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,"Country_Bahamas, The",Country_Bangladesh,Country_Belarus,...,Country_Tunisia,Country_Turkey,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States,"Country_Venezuela, RB",Country_Vietnam,Country_Zimbabwe,Country_india
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6539,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Fazendo a escala para as colunas numéricas originais

In [ ]:
#Initialize our scaler
scaler = StandardScaler()

In [ ]:
# Salvando os nomes originais das colunas para renomeação depois, assim como o autor fez
nomesColOriginais = numer.columns

# fazendo a escala
numer = pd.DataFrame(scaler.fit_transform(numer))

In [ ]:
#renaming cols that has been scaled
numer.columns = [f"{nomesColOriginais[i]}_scaled" for i in range(numer.shape[1])]
numer

,Raised_scaled,ESG_scaled,E_scaled,S_scaled,G_scaled
0,0.086857,3.092590,4.880242,0.769739,-0.229437
1,-0.187503,5.809621,6.126549,2.782878,1.249132
2,-0.027090,4.451105,6.126549,-0.236831,2.727700
3,-0.187503,2.413333,3.633935,-0.236831,1.249132
4,-0.187503,5.469992,-0.104986,5.299302,4.206269
...,...,...,...,...,...
6536,-0.187503,-0.303698,-0.104986,-0.236831,-0.229437
6537,-0.187503,-0.303698,-0.104986,-0.236831,-0.229437
6538,-0.187503,-0.303698,-0.104986,-0.236831,-0.229437
6539,-0.187503,-0.303698,-0.104986,-0.236831,-0.229437


In [ ]:
# Unindo os dataframes numer e cater, mas agora com uma escala aplicada em numer
X = pd.concat([cater, numer], axis=1, join='inner')
X.head()

,Country_Afghanistan,Country_Albania,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,"Country_Bahamas, The",Country_Bangladesh,Country_Belarus,...,Country_United States,"Country_Venezuela, RB",Country_Vietnam,Country_Zimbabwe,Country_india,Raised_scaled,ESG_scaled,E_scaled,S_scaled,G_scaled
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.086857,3.092590,4.880242,0.769739,-0.229437
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.187503,5.809621,6.126549,2.782878,1.249132
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,-0.027090,4.451105,6.126549,-0.236831,2.727700
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.187503,2.413333,3.633935,-0.236831,1.249132
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.187503,5.469992,-0.104986,5.299302,4.206269


### Time to build our clusters

In [ ]:
#Initialize our model
kmeans = KMeans(n_clusters=4, random_state= 608)

In [ ]:
#Fit our model
kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=4, random_state=608)

In [ ]:
#Find which cluster each data-point belongs to
clusters = kmeans.predict(X)

In [ ]:
#Add the cluster vector to our DataFrame, X
X["Cluster"] = clusters

## PCA (Principal Components Analysis)

But first, we will create a seperate, smaller DataFrame, plotX, to plot our data with. The reason we create a smaller DataFrame is so that we can plot our data faster, and so that our plots do not turn out looking too messy or over-crowded.

In [ ]:
#plotX is a DataFrame containing 5000 values sampled randomly from X
#plotX = pd.DataFrame(np.array(X.sample(5000))) # original do tutorial
plotX = X

#Rename plotX's columns since it was briefly converted to an np.array above
plotX.columns = X.columns

initialize our PCA models

In [ ]:
#PCA with two principal components
pca_2d = PCA(n_components=2)

build our new DataFrames

In [ ]:
#This DataFrame contains the two principal components that will be used
#for the 2-D visualization mentioned above
PCs_2d = pd.DataFrame(pca_2d.fit_transform(plotX.drop(["Cluster"], axis=1)))

Rename the columns of these newly created DataFrames

In [ ]:
PCs_2d.columns = ["PC1_2d", "PC2_2d"]

We concatenate these newly created DataFrames to plotX so that they can be used by plotX as columns

In [ ]:
plotX = pd.concat([plotX,PCs_2d], axis=1, join='inner')

now we divide our DataFrame, plotX, into three new DataFrames.

Each of these new DataFrames will hold all of the values contained in exacltly one of the clusters. For example, all of the values contained within the DataFrame, cluster0 will belong to 'cluster 0', and all the values contained in DataFrame, cluster1 will belong to 'cluster 1', etc.

In [ ]:
#Note that all of the DataFrames below are sub-DataFrames of 'plotX'.
#This is because we intend to plot the values contained within each of these DataFrames.

cluster0 = plotX[plotX["Cluster"] == 0]
cluster1 = plotX[plotX["Cluster"] == 1]
cluster2 = plotX[plotX["Cluster"] == 2]
cluster3 = plotX[plotX["Cluster"] == 3]

### PCA Visualizations:

In [ ]:
# Plot para 'Cluster 0'
trace0 = go.Scatter(x=cluster0["PC1_2d"], y=cluster0["PC2_2d"], mode='markers', name='Cluster 0', marker=dict(color='rgba(255, 128, 255, 0.8)'))

# Plot para 'Cluster 1'
trace1 = go.Scatter(x=cluster1["PC1_2d"], y=cluster1["PC2_2d"], mode='markers', name='Cluster 1', marker=dict(color='rgba(255, 128, 25, 0.8)'))

# Plot para 'Cluster 2'
trace2 = go.Scatter(x=cluster2["PC1_2d"], y=cluster2["PC2_2d"], mode='markers', name='Cluster 2', marker=dict(color='rgba(0, 255, 200, 0.8)'))

# Plot para 'Cluster 3'
trace3 = go.Scatter(x=cluster3["PC1_2d"], y=cluster3["PC2_2d"], mode='markers', name='Cluster 3', marker=dict(color='rgba(255, 0, 0, 0.8)'))

# Configurações do gráfico
layout = go.Layout(title="Clusters gerados ", xaxis=dict(title="PC1"), yaxis=dict(title="PC2"))
fig = go.Figure(data=[trace0, trace1, trace2, trace3], layout=layout)

# Mostrar o gráfico
fig.show()
